<a href="https://colab.research.google.com/github/FedeYB/Laboratorio_1-/blob/main/Laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conexión con el drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Librerias


In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np

#Limpieza del csv

In [ ]:
ruta_de_acceso = '/content/drive/MyDrive/UPSO/UPSO/Segundo año/Segundo cuatrimestre/Elemento de aprendizaje de maquina y big data/Clase 3/Dataset de ventas2'

archivos = os.listdir(ruta_de_acceso)

dataframes = {}# este diccionario sirve para dsp guardar el nombre del mes con el dataframe del mes
               # ej: 'Enero': Df_enero

for archivo in archivos:
  if archivo.endswith('.csv'):
    #mes = archivo.split('_')[3] # te sepeara por cada '_' que encuentre, y te devuelve el [3] elemento
    mes = archivo.split('_')[3].split('.')[0] # le agregamos el "split('.')[0]" para eliminar el ".csv" y obtener solo el nombre del mes
    ruta_completa = os.path.join(ruta_de_acceso, archivo) # con esto juntamos la ruta de la cartepeta principalcon el nombre del archivo para obtener la ruta completa
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy() # hacemos una copia del df para no modificar el original

    # transformo todos lo valores no numericos en NaN en las columnas 'Cantidad Pedida' y 'Precio Unitario'
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')

    # Elimino todos los NaN y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()

    # Transformo todos los valores numericos al tipo de dato int y float
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)

    dataframes[mes] = df_ventas # guardamos el mes y el df del mes en el diccionario
    # Habria que ordenar los df dentro de el diccionario


#Funcion para extraer los datos del DataFrame

In [ ]:
def extraer_datos(datos):
  datos['Fecha de pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de pedido'].dt.hour
  datos['Dia'] = datos['Fecha de pedido'].dt.day
  datos['Dia de la semana'] = datos['Fecha de pedido'].dt.weekday
  return datos


#Pregunta 5


##5. Tendencia ventas:<br>
● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?<br>
● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?<br>
● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

###● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días en los diferentes meses?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias = list(range(1,32))
color_index = 0
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#FF5733', '#33FF57']

fig = go.Figure()

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia = dataframe_mes.groupby(['Dia'])['Cantidad Pedida'].sum()

  max_valor = Ventas_por_dia.max()

  markers = []

  for val in Ventas_por_dia:
    if val == max_valor:
      markers.append(val)
    else:
      markers.append(None)

  color_linea = colores[color_index]

  fig.add_trace(go.Scatter(

    x=Dias,
    y=Ventas_por_dia,
    mode='lines',
    name=mes,
    line=dict(shape='spline', smoothing=0.8, width=2, color=color_linea)
    )
  )

  fig.add_trace(go.Scatter(
        x=Dias,
        y=markers,
        mode='markers',
        name=mes,
        marker=dict(symbol='circle', size=10, opacity=1, color=color_linea),
        showlegend=False
    ))
  color_index += 1

fig.update_layout(

    title="Unidades vendidas por dia en el mes",
    xaxis=dict(title="Dias del mes", tickvals=Dias),
    yaxis=dict(title="Unidades vendidas", side="left"))


fig.show()

###● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
markers = []
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#FF5733', '#33FF57']
color_index = 0
fig = go.Figure()

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia_de_semana = dataframe_mes.groupby(['Dia de la semana'])['Cantidad Pedida'].sum()

  max_valor = Ventas_por_dia_de_semana.max()

  markers = []

  for val in Ventas_por_dia_de_semana:
    if val == max_valor:
      markers.append(val)
    else:
      markers.append(None)

  color_linea = colores[color_index]

  fig.add_trace(go.Scatter(

      x=Dias_de_la_semana,
      y=Ventas_por_dia_de_semana,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=0.8, width=2, color=color_linea),
      text=markers
      ))

  fig.add_trace(go.Scatter(
        x=Dias_de_la_semana,
        y=markers,
        mode='markers',
        name=mes,
        marker=dict(symbol='circle', size=10, opacity=1, color=color_linea),
        showlegend=False
    ))

  color_index += 1

fig.update_layout(

    title="Unidades vendidas por dia de semana",
    xaxis=dict(title="Dias de la semana", tickvals=Dias_de_la_semana),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()


###● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y los fines de semana?

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Dias_de_la_semana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
mediana_dias = []
Lunes = []
Martes = []
Miercoles = []
Jueves = []
Viernes = []
Sabado = []
Domingo = []

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframe_mes = extraer_datos(dataframe_mes)
  Ventas_por_dia_de_semana = dataframe_mes.groupby(['Dia de la semana'])['Cantidad Pedida'].sum()
  Lunes.append(Ventas_por_dia_de_semana[0])
  Martes.append(Ventas_por_dia_de_semana[1])
  Miercoles.append(Ventas_por_dia_de_semana[2])
  Jueves.append(Ventas_por_dia_de_semana[3])
  Viernes.append(Ventas_por_dia_de_semana[4])
  Sabado.append(Ventas_por_dia_de_semana[5])
  Domingo.append(Ventas_por_dia_de_semana[6])

mediana_dias.append(np.median(Lunes))
mediana_dias.append(np.median(Martes))
mediana_dias.append(np.median(Miercoles))
mediana_dias.append(np.median(Jueves))
mediana_dias.append(np.median(Viernes))
mediana_dias.append(np.median(Sabado))
mediana_dias.append(np.median(Domingo))

colores = ['blue', 'orange', 'green', 'red', 'purple', 'burlywood', 'pink']

#Creo el objeto figura
fig = go.Figure()

#Agrego grafico en mi objeto figura
fig.add_trace(go.Bar(
    x = Dias_de_la_semana,
    y = mediana_dias,
    name = 'Unidades Vendidas',
    marker = dict(color = colores),
    )
)

fig.update_layout(
    title = 'Mediana de las unidades vendidas por dia de la semana',
    xaxis = dict(title = 'Dias de la semana'),
    yaxis = dict(title = 'Mediana de las unidades vendidas', side = 'left')
)
